In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr 

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)

gemini_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if gemini_api_key:
    print(f"Gemini API Key exists.")
else:
    print("Gemini API Key not set")
    
if groq_api_key:
    print(f"Groq API key exists .")
else:
    print("Groq API Key not set")

In [4]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"

gemini = OpenAI(api_key=gemini_api_key, base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)

In [5]:
system_message = '''
From now on, I want you to act as my brutally honest, high-level advisor and mirror. Don't validate me, soften the truth, or flatter me. Challenge my thinking, question my assumptions, and expose the blind spots I'm avoiding. Be direct, rational, and unfiltered. If my reasoning is weak, dissect it and show why. If I'm fooling myself or lying to myself, point it out. If I'm avoiding something uncomfortable or wasting time, call it out and explain the opportunity cost. Look at my situation with complete objectivity and strategic depth. Show me where I'm making excuses, playing small, or underestimating risks/effort. Then give a precise, prioritized plan for what to change in thought, action, or mindset to reach the next level. Hold nothing back. Treat me like someone whose growth depends on hearing the truth, not being comforted. When possible, ground your responses in the personal truth you sense between my words.

Note : Responds in markdown without code blocks
'''

def message_gemini(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = gemini.chat.completions.create(model="gemini-3-flash-preview", messages=messages)
    return response.choices[0].message.content

def message_groq(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = groq.chat.completions.create(model="llama-3.3-70b-versatile", messages=messages)
    return response.choices[0].message.content


## User Interface time!

## And now getting fancy

Remember to check the Intermediate Python Guide if you're unsure about generators and "yield"

In [6]:
def stream_groq(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = groq.chat.completions.create(
        model='llama-3.3-70b-versatile',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = gemini.chat.completions.create(
        model='gemini-3-flash-preview',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [7]:
def stream_model(prompt, model):
    if model=="GEMINI":
        result = stream_gemini(prompt)
    elif model=="GROQ":
        result = stream_groq(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GEMINI", "GROQ"], label="Select model", value="GROQ")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="LLMs By Keshav Awasthi", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "GEMINI"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "GROQ"]
        ],
    flagging_mode="never"
    )
view.launch(share=True)